In [67]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
from pyspark.sql.functions import col

In [68]:
spark = SparkSession.builder.getOrCreate()

In [69]:
data_path = "datos/sample_10e3.json"
df = spark.read.json(data_path)
df.columns

['_id',
 'ageRange',
 'idplug_base',
 'idplug_station',
 'idunplug_base',
 'idunplug_station',
 'travel_time',
 'unplug_hourTime',
 'user_day_code',
 'user_type',
 'zip_code']

### Uso medio

Voy a hacer la comparativa de tiempo de uso por segmentos, comparando el número de individuos que superan la media en cada grupo, esto esta sesgado por si un grupo lo usa más, para corregirlo lo podemos hacer por porcentajes.

In [70]:
df_1 = df.groupby('ageRange').count().orderBy('ageRange')
df_1.show()

+--------+-----+
|ageRange|count|
+--------+-----+
|       0|  491|
|       1|   19|
|       2|   12|
|       3|   75|
|       4|  267|
|       5|  131|
|       6|    5|
+--------+-----+



Problema -> La mayor parte de la gente esta en grupo 0, desconocemos su edad.

In [71]:
df_media     = df.select('travel_time')
df_media.show(5)

+-----------+
|travel_time|
+-----------+
|        219|
|        359|
|        375|
|        264|
|        367|
+-----------+
only showing top 5 rows



In [72]:
elementos    = df_media.count()
suma         = df_media.groupBy().sum().collect()[0][0]
average      = suma//elementos

In [73]:
average
# Un poco alto no?

904

In [74]:
df_3    = df.select('ageRange', 'travel_time')
df_3.show(5)

+--------+-----------+
|ageRange|travel_time|
+--------+-----------+
|       0|        219|
|       4|        359|
|       4|        375|
|       5|        264|
|       4|        367|
+--------+-----------+
only showing top 5 rows



In [77]:
udf_resta_media = udf(lambda x: x - average, IntegerType())
df_4 = df_3.withColumn('travel_time', udf_resta_media('travel_time'))


In [79]:
df_4.show(5)

+--------+-----------+
|ageRange|travel_time|
+--------+-----------+
|       0|       -685|
|       4|       -545|
|       4|       -529|
|       5|       -640|
|       4|       -537|
+--------+-----------+
only showing top 5 rows



In [81]:
df_4 = df_4.filter(col('travel_time') > 0)
df_4.show(5)

+--------+-----------+
|ageRange|travel_time|
+--------+-----------+
|       0|         83|
|       4|        142|
|       4|          1|
|       5|        108|
|       6|        132|
+--------+-----------+
only showing top 5 rows



In [84]:
df_resultado = df_4.groupby('ageRange').count().orderBy('ageRange')
df_resultado.show()

+--------+-----+
|ageRange|count|
+--------+-----+
|       0|  172|
|       1|   13|
|       2|    4|
|       3|   20|
|       4|   91|
|       5|   45|
|       6|    4|
+--------+-----+



Falta ajustarlo al porcentaje del tamaño del grupo